In [2]:
# import json
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModel
# from tqdm import tqdm
# from sklearn.model_selection import train_test_split
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import pandas as pd

# # ====== 配置 ======
# MODEL_NAME = "microsoft/deberta-v3-base"
# MAX_LEN = 256
# BATCH_SIZE = 4
# EPOCHS = 5
# LR = 2e-5
# LAMBDA_CL = 1.0   # 对比学习权重
# SAVE_PATH = "best_deberta_contrastive.pt"
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ====== 工具函数 ======
# def mbti_to_binary(mbti_type):
#     return {
#         "ei": 0 if mbti_type[0] == "I" else 1,
#         "ns": 0 if mbti_type[1] == "S" else 1,
#         "tf": 0 if mbti_type[2] == "F" else 1,
#         "jp": 0 if mbti_type[3] == "P" else 1
#     }

# # ====== Dataset ======
# class JointMBTIDataset(Dataset):
#     def __init__(self, data, tokenizer, max_length=256):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def _encode(self, text):
#         return self.tokenizer(
#             text,
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_length,
#             return_tensors="pt"
#         )

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         base_text = item.get("posts_cleaned", item["posts"]) or ""
#         semantic = item.get("semantic_view", "")
#         sentiment = item.get("sentiment_view", "")
#         linguistic = item.get("linguistic_view", "")

#         mbti_label = mbti_to_binary(item["type"])

#         enc_base = self._encode(base_text)
#         enc_sem = self._encode(semantic or base_text)
#         enc_sent = self._encode(sentiment or base_text)
#         enc_ling = self._encode(linguistic or base_text)

#         return {
#             # 原文
#             "input_ids": enc_base["input_ids"].squeeze(0),
#             "attention_mask": enc_base["attention_mask"].squeeze(0),
#             # 三视角
#             "semantic_ids": enc_sem["input_ids"].squeeze(0),
#             "semantic_mask": enc_sem["attention_mask"].squeeze(0),
#             "sentiment_ids": enc_sent["input_ids"].squeeze(0),
#             "sentiment_mask": enc_sent["attention_mask"].squeeze(0),
#             "linguistic_ids": enc_ling["input_ids"].squeeze(0),
#             "linguistic_mask": enc_ling["attention_mask"].squeeze(0),
#             # 标签
#             "ei": torch.tensor(mbti_label["ei"], dtype=torch.float),
#             "ns": torch.tensor(mbti_label["ns"], dtype=torch.float),
#             "tf": torch.tensor(mbti_label["tf"], dtype=torch.float),
#             "jp": torch.tensor(mbti_label["jp"], dtype=torch.float),
#             "label": item["type"]  # 用于可视化
#         }

# # ====== 模型 ======
# class JointMBTIModel(nn.Module):
#     def __init__(self, encoder_name=MODEL_NAME, dropout=0.1):
#         super().__init__()
#         self.encoder = AutoModel.from_pretrained(encoder_name)
#         hidden_size = self.encoder.config.hidden_size
#         self.dropout = nn.Dropout(dropout)
#         self.classifier_ei = nn.Linear(hidden_size, 1)
#         self.classifier_ns = nn.Linear(hidden_size, 1)
#         self.classifier_tf = nn.Linear(hidden_size, 1)
#         self.classifier_jp = nn.Linear(hidden_size, 1)

#     def get_embedding(self, input_ids, attention_mask):
#         out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
#         cls_emb = self.dropout(out.last_hidden_state[:, 0, :])
#         return cls_emb

#     def forward(self, input_ids, attention_mask):
#         cls_emb = self.get_embedding(input_ids, attention_mask)
#         return {
#             "embedding": cls_emb,
#             "ei": self.classifier_ei(cls_emb),
#             "ns": self.classifier_ns(cls_emb),
#             "tf": self.classifier_tf(cls_emb),
#             "jp": self.classifier_jp(cls_emb)
#         }

# # ====== 损失函数 ======
# def compute_mbti_loss(preds, labels):
#     bce = nn.BCEWithLogitsLoss()
#     return (
#         bce(preds["ei"].squeeze(), labels["ei"]) +
#         bce(preds["ns"].squeeze(), labels["ns"]) +
#         bce(preds["tf"].squeeze(), labels["tf"]) +
#         bce(preds["jp"].squeeze(), labels["jp"])
#     ) / 4

# def contrastive_loss(anchor, positives, temperature=0.07):
#     anchor = F.normalize(anchor, dim=1)
#     losses = []
#     for pos in positives:
#         pos = F.normalize(pos, dim=1)
#         logits = torch.matmul(anchor, pos.T) / temperature
#         labels = torch.arange(anchor.size(0), device=anchor.device)
#         losses.append(F.cross_entropy(logits, labels))
#     return sum(losses) / len(losses)

# # ====== 训练/评估 ======
# def train_model(model, train_loader, val_loader, optimizer, device, epochs=5, save_path="best.pt"):
#     model.to(device)
#     best_val_acc_all = 0.0

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0

#         for batch in tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{epochs}"):
#             # 输入
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             semantic_ids = batch["semantic_ids"].to(device)
#             semantic_mask = batch["semantic_mask"].to(device)
#             sentiment_ids = batch["sentiment_ids"].to(device)
#             sentiment_mask = batch["sentiment_mask"].to(device)
#             linguistic_ids = batch["linguistic_ids"].to(device)
#             linguistic_mask = batch["linguistic_mask"].to(device)

#             labels = {k: batch[k].to(device) for k in ["ei", "ns", "tf", "jp"]}

#             optimizer.zero_grad()

#             # 分类损失
#             out_base = model(input_ids=input_ids, attention_mask=attention_mask)
#             loss_cls = compute_mbti_loss(out_base, labels)

#             # 对比损失
#             emb_base = out_base["embedding"]
#             emb_sem = model.get_embedding(semantic_ids, semantic_mask)
#             emb_sent = model.get_embedding(sentiment_ids, sentiment_mask)
#             emb_ling = model.get_embedding(linguistic_ids, linguistic_mask)
#             loss_cl = contrastive_loss(emb_base, [emb_sem, emb_sent, emb_ling])

#             loss = loss_cls + LAMBDA_CL * loss_cl
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#         avg_loss = total_loss / len(train_loader)
#         print(f"\nEpoch {epoch+1} - Train Loss: {avg_loss:.4f}")

#         # 验证（返回各维准确率 + 4D）
#         metrics = evaluate(model, val_loader, device)
#         print(
#             "Validation Accuracy | "
#             f"EI: {metrics['acc_ei']:.2%}  "
#             f"NS: {metrics['acc_ns']:.2%}  "
#             f"TF: {metrics['acc_tf']:.2%}  "
#             f"JP: {metrics['acc_jp']:.2%}  "
#             f"4D: {metrics['acc_all']:.2%}"
#         )

#         # 保存最优（按 4D match）
#         if metrics["acc_all"] > best_val_acc_all:
#             best_val_acc_all = metrics["acc_all"]
#             torch.save(model.state_dict(), save_path)
#             print(f"✅ Best model saved to {save_path} (4D match: {best_val_acc_all:.2%})")

# def evaluate(model, dataloader, device):
#     model.eval()
#     total = 0
#     correct_ei = correct_ns = correct_tf = correct_jp = correct_all = 0

#     with torch.no_grad():
#         for batch in dataloader:
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             labels = {k: batch[k].to(device) for k in ["ei", "ns", "tf", "jp"]}

#             outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#             pred_ei = torch.sigmoid(outputs["ei"]).round().squeeze()
#             pred_ns = torch.sigmoid(outputs["ns"]).round().squeeze()
#             pred_tf = torch.sigmoid(outputs["tf"]).round().squeeze()
#             pred_jp = torch.sigmoid(outputs["jp"]).round().squeeze()

#             correct_ei += (pred_ei == labels["ei"]).sum().item()
#             correct_ns += (pred_ns == labels["ns"]).sum().item()
#             correct_tf += (pred_tf == labels["tf"]).sum().item()
#             correct_jp += (pred_jp == labels["jp"]).sum().item()

#             correct_all += (
#                 (pred_ei == labels["ei"]) &
#                 (pred_ns == labels["ns"]) &
#                 (pred_tf == labels["tf"]) &
#                 (pred_jp == labels["jp"])
#             ).sum().item()

#             total += input_ids.size(0)

#     acc_ei = correct_ei / total
#     acc_ns = correct_ns / total
#     acc_tf = correct_tf / total
#     acc_jp = correct_jp / total
#     acc_all = correct_all / total

#     return {
#         "acc_ei": acc_ei,
#         "acc_ns": acc_ns,
#         "acc_tf": acc_tf,
#         "acc_jp": acc_jp,
#         "acc_all": acc_all
#     }

# # ====== 可视化（t-SNE）======
# @torch.no_grad()
# def visualize_embeddings(model, dataloader, device, title="MBTI Embedding Space (t-SNE)"):
#     model.eval()
#     all_embeds, all_labels = [], []

#     for batch in dataloader:
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         embs = model.get_embedding(input_ids, attention_mask)
#         all_embeds.append(embs.cpu())
#         all_labels.extend(batch["label"])  # MBTI 字符串类型

#     embeddings = torch.cat(all_embeds, dim=0).numpy()
#     tsne = TSNE(n_components=2, random_state=42, init="pca", learning_rate="auto")
#     xy = tsne.fit_transform(embeddings)

#     df = pd.DataFrame(xy, columns=["x", "y"])
#     df["label"] = all_labels

#     plt.figure(figsize=(10, 8))
#     for label in sorted(set(all_labels)):
#         pts = df[df["label"] == label]
#         plt.scatter(pts["x"], pts["y"], alpha=0.65, s=18, label=label)
#     plt.legend(markerscale=1.5, bbox_to_anchor=(1.05, 1), loc="upper left", ncol=1)
#     plt.title(title)
#     plt.tight_layout()
#     plt.show()


In [3]:
# if __name__ == "__main__":
#     with open("mbti_sample_with_all_views.json", "r", encoding="utf-8") as f:
#         full_data = json.load(f)

#     train_data, val_data = train_test_split(full_data, test_size=0.2, random_state=42)

#     tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#     train_dataset = JointMBTIDataset(train_data, tokenizer, max_length=MAX_LEN)
#     val_dataset = JointMBTIDataset(val_data, tokenizer, max_length=MAX_LEN)

#     train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#     model = JointMBTIModel(MODEL_NAME)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

#     # 训练
#     train_model(model, train_loader, val_loader, optimizer, DEVICE, epochs=50, save_path=SAVE_PATH)

#     # 可视化（用验证集）
#     print("\n📊 Visualizing embeddings on the validation set...")
#     visualize_embeddings(model, val_loader, DEVICE, title="MBTI CLS Embedding Space (t-SNE)")


In [ ]:
#TYB模型
# import json
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModel
# from tqdm import tqdm
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import pandas as pd
# import random

# # ====== 配置 ======
# MODEL_NAME = "microsoft/deberta-v3-base"
# MAX_LEN = 256
# BATCH_SIZE = 4
# EPOCHS = 30
# LR = 2e-5
# LAMBDA_CL = 0.1   # 对比学习权重
# SAVE_PATH = "best_deberta_supcon.pt"
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ====== 工具函数 ======
# def mbti_to_binary(mbti_type):
#     return {
#         "ei": 0 if mbti_type[0] == "I" else 1,
#         "ns": 0 if mbti_type[1] == "S" else 1,
#         "tf": 0 if mbti_type[2] == "F" else 1,
#         "jp": 0 if mbti_type[3] == "P" else 1
#     }

# def mbti_to_classid(mbti_type):
#     """将16种MBTI映射为0-15整数标签"""
#     types = [
#         "INTJ","INTP","ENTJ","ENTP",
#         "INFJ","INFP","ENFJ","ENFP",
#         "ISTJ","ISFJ","ESTJ","ESFJ",
#         "ISTP","ISFP","ESTP","ESFP"
#     ]
#     return types.index(mbti_type)

# def random_drop_words(text, drop_prob=0.1):
#     """随机丢弃部分token，防止过拟合explain提示词"""
#     words = text.split()
#     keep = [w for w in words if random.random() > drop_prob]
#     return " ".join(keep) if keep else text

# # ====== Dataset ======
# class JointMBTIDataset(Dataset):
#     def __init__(self, data, tokenizer, max_length=256, drop_prob=0.1):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.drop_prob = drop_prob

#     def _encode(self, text):
#         return self.tokenizer(
#             text,
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_length,
#             return_tensors="pt"
#         )

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         base_text = item.get("posts_cleaned", item["posts"]) or ""

#         # 三视角（加随机DropWord）
#         semantic = random_drop_words(item.get("semantic_view", ""), self.drop_prob)
#         sentiment = random_drop_words(item.get("sentiment_view", ""), self.drop_prob)
#         linguistic = random_drop_words(item.get("linguistic_view", ""), self.drop_prob)

#         mbti_label = mbti_to_binary(item["type"])
#         class_id = mbti_to_classid(item["type"])

#         enc_base = self._encode(base_text)
#         enc_sem = self._encode(semantic or base_text)
#         enc_sent = self._encode(sentiment or base_text)
#         enc_ling = self._encode(linguistic or base_text)

#         return {
#             # 主文本
#             "input_ids": enc_base["input_ids"].squeeze(0),
#             "attention_mask": enc_base["attention_mask"].squeeze(0),
#             # 三视角
#             "semantic_ids": enc_sem["input_ids"].squeeze(0),
#             "semantic_mask": enc_sem["attention_mask"].squeeze(0),
#             "sentiment_ids": enc_sent["input_ids"].squeeze(0),
#             "sentiment_mask": enc_sent["attention_mask"].squeeze(0),
#             "linguistic_ids": enc_ling["input_ids"].squeeze(0),
#             "linguistic_mask": enc_ling["attention_mask"].squeeze(0),
#             # 二进制标签
#             "ei": torch.tensor(mbti_label["ei"], dtype=torch.float),
#             "ns": torch.tensor(mbti_label["ns"], dtype=torch.float),
#             "tf": torch.tensor(mbti_label["tf"], dtype=torch.float),
#             "jp": torch.tensor(mbti_label["jp"], dtype=torch.float),
#             # 分类id
#             "class_id": torch.tensor(class_id, dtype=torch.long),
#             "label": item["type"]
#         }

# # ====== 模型 ======
# class JointMBTIModel(nn.Module):
#     def __init__(self, encoder_name=MODEL_NAME, dropout=0.3):
#         super().__init__()
#         self.encoder = AutoModel.from_pretrained(encoder_name)
#         hidden_size = self.encoder.config.hidden_size
#         self.dropout = nn.Dropout(dropout)
#         self.classifier_ei = nn.Linear(hidden_size, 1)
#         self.classifier_ns = nn.Linear(hidden_size, 1)
#         self.classifier_tf = nn.Linear(hidden_size, 1)
#         self.classifier_jp = nn.Linear(hidden_size, 1)

#     def get_embedding(self, input_ids, attention_mask):
#         out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
#         cls_emb = self.dropout(out.last_hidden_state[:, 0, :])
#         return cls_emb

#     def forward(self, input_ids, attention_mask):
#         cls_emb = self.get_embedding(input_ids, attention_mask)
#         return {
#             "embedding": cls_emb,
#             "ei": self.classifier_ei(cls_emb),
#             "ns": self.classifier_ns(cls_emb),
#             "tf": self.classifier_tf(cls_emb),
#             "jp": self.classifier_jp(cls_emb)
#         }

# # ====== 损失函数 ======
# def compute_mbti_loss(preds, labels):
#     bce = nn.BCEWithLogitsLoss()
#     return (
#         bce(preds["ei"].squeeze(), labels["ei"]) +
#         bce(preds["ns"].squeeze(), labels["ns"]) +
#         bce(preds["tf"].squeeze(), labels["tf"]) +
#         bce(preds["jp"].squeeze(), labels["jp"])
#     ) / 4

# def supervised_contrastive_loss(embeddings, labels, temperature=0.07):
#     """Supervised Contrastive Loss"""
#     device = embeddings.device
#     embeddings = F.normalize(embeddings, dim=1)
#     similarity = torch.matmul(embeddings, embeddings.T) / temperature

#     mask = torch.eye(len(labels), dtype=torch.bool, device=device)
#     labels = labels.contiguous().view(-1, 1)
#     match_mask = torch.eq(labels, labels.T).to(device) & ~mask

#     logits_max, _ = torch.max(similarity, dim=1, keepdim=True)
#     logits = similarity - logits_max.detach()
#     exp_logits = torch.exp(logits) * (~mask)
#     log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

#     mean_log_prob_pos = (match_mask * log_prob).sum(1) / match_mask.sum(1).clamp(min=1)
#     loss = -mean_log_prob_pos.mean()
#     return loss

# # ====== 训练 ======
# def train_model(model, train_loader, val_loader, optimizer, device, epochs=5, save_path="best.pt"):
#     model.to(device)
#     best_val_acc_all = 0.0

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0

#         for batch in tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{epochs}"):
#             optimizer.zero_grad()

#             # 主文本
#             out_base = model(batch["input_ids"].to(device), batch["attention_mask"].to(device))
#             loss_cls = compute_mbti_loss(out_base, {k: batch[k].to(device) for k in ["ei","ns","tf","jp"]})

#             # 三视角 embedding
#             emb_list = [out_base["embedding"]]
#             label_list = [batch["class_id"].to(device)]

#             for view in ["semantic", "sentiment", "linguistic"]:
#                 emb = model.get_embedding(batch[f"{view}_ids"].to(device), batch[f"{view}_mask"].to(device))
#                 emb_list.append(emb)
#                 label_list.append(batch["class_id"].to(device))

#             # 拼接所有 embedding
#             all_embeddings = torch.cat(emb_list, dim=0)
#             all_labels = torch.cat(label_list, dim=0)

#             loss_cl = supervised_contrastive_loss(all_embeddings, all_labels)
#             loss = loss_cls + LAMBDA_CL * loss_cl

#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#             optimizer.step()
#             total_loss += loss.item()

#         avg_loss = total_loss / len(train_loader)
#         print(f"\nEpoch {epoch+1} - Train Loss: {avg_loss:.4f}")

#         metrics = evaluate(model, val_loader, device)
#         print(f"Validation | EI: {metrics['acc_ei']:.2%} NS: {metrics['acc_ns']:.2%} TF: {metrics['acc_tf']:.2%} JP: {metrics['acc_jp']:.2%} 4D: {metrics['acc_all']:.2%}")

#         if metrics["acc_all"] > best_val_acc_all:
#             best_val_acc_all = metrics["acc_all"]
#             torch.save(model.state_dict(), save_path)
#             print(f"✅ Best model saved to {save_path} (4D: {best_val_acc_all:.2%})")

# # ====== 评估 ======
# def evaluate(model, dataloader, device):
#     model.eval()
#     total = 0
#     correct_ei = correct_ns = correct_tf = correct_jp = correct_all = 0

#     with torch.no_grad():
#         for batch in dataloader:
#             outputs = model(batch["input_ids"].to(device), batch["attention_mask"].to(device))
#             labels = {k: batch[k].to(device) for k in ["ei", "ns", "tf", "jp"]}

#             pred_ei = torch.sigmoid(outputs["ei"]).round().squeeze()
#             pred_ns = torch.sigmoid(outputs["ns"]).round().squeeze()
#             pred_tf = torch.sigmoid(outputs["tf"]).round().squeeze()
#             pred_jp = torch.sigmoid(outputs["jp"]).round().squeeze()

#             correct_ei += (pred_ei == labels["ei"]).sum().item()
#             correct_ns += (pred_ns == labels["ns"]).sum().item()
#             correct_tf += (pred_tf == labels["tf"]).sum().item()
#             correct_jp += (pred_jp == labels["jp"]).sum().item()

#             correct_all += ((pred_ei == labels["ei"]) &
#                             (pred_ns == labels["ns"]) &
#                             (pred_tf == labels["tf"]) &
#                             (pred_jp == labels["jp"])).sum().item()

#             total += batch["input_ids"].size(0)

#     return {
#         "acc_ei": correct_ei / total,
#         "acc_ns": correct_ns / total,
#         "acc_tf": correct_tf / total,
#         "acc_jp": correct_jp / total,
#         "acc_all": correct_all / total
#     }


In [6]:
# ====================== 最优版整体代码（含自定义 collate_fn） ======================
import os, json, random
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 可选：静默 tokenizer 并行告警

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
)
from torch import amp
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import contextlib

# ===================== 配置 =====================
MODEL_NAME   = "microsoft/deberta-v3-base"
FILE_PATH    = "mbti_sample_with_all_views_pandora.json"
MAX_LEN      = 320        # 建议：320；想更快可改 288 做 A/B
BATCH_SIZE   = 12         # 结合 MAX_LEN 的建议 batch
GRAD_ACCUM   = 2          # 梯度累积，保证有效 tokens/step 足够大
EPOCHS       = 30
LR           = 2e-5
WARMUP_RATIO = 0.06
LAMBDA_CL    = 0.1        # 对比学习权重
SAVE_PATH    = "best_deberta_supcon.pt"
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===================== 工具函数 =====================
def mbti_to_binary(mbti_type):
    return {
        "ei": 0 if mbti_type[0] == "I" else 1,
        "ns": 0 if mbti_type[1] == "S" else 1,
        "tf": 0 if mbti_type[2] == "F" else 1,
        "jp": 0 if mbti_type[3] == "P" else 1
    }

def mbti_to_classid(mbti_type):
    types = [
        "INTJ","INTP","ENTJ","ENTP",
        "INFJ","INFP","ENFJ","ENFP",
        "ISTJ","ISFJ","ESTJ","ESFJ",
        "ISTP","ISFP","ESTP","ESFP"
    ]
    return types.index(mbti_type)

def random_drop_words(text, drop_prob=0.1):
    words = text.split()
    keep = [w for w in words if random.random() > drop_prob]
    return " ".join(keep) if keep else text

# ===================== Dataset（只截断，不在此处padding） =====================
class JointMBTIDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=320, drop_prob=0.1):
        self.data = data
        self.tok = tokenizer
        self.max_length = max_length
        self.drop_prob = drop_prob

    def _encode(self, text):
        enc = self.tok(text, truncation=True, max_length=self.max_length)
        return enc["input_ids"], enc["attention_mask"]

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        base_text = item.get("posts_cleaned", item.get("posts", "")) or ""

        # 三视角（随机DropWord增强）
        semantic   = random_drop_words(item.get("semantic_view", ""), self.drop_prob) or base_text
        sentiment  = random_drop_words(item.get("sentiment_view",""), self.drop_prob) or base_text
        linguistic = random_drop_words(item.get("linguistic_view",""), self.drop_prob) or base_text

        mbti_label = mbti_to_binary(item["type"])
        class_id   = mbti_to_classid(item["type"])

        base_ids, base_m = self._encode(base_text)
        sem_ids,  sem_m  = self._encode(semantic)
        sen_ids,  sen_m  = self._encode(sentiment)
        lin_ids,  lin_m  = self._encode(linguistic)

        return {
            "input_ids": base_ids, "attention_mask": base_m,
            "semantic_ids": sem_ids, "semantic_mask": sem_m,
            "sentiment_ids": sen_ids, "sentiment_mask": sen_m,
            "linguistic_ids": lin_ids, "linguistic_mask": lin_m,
            "ei": torch.tensor(mbti_label["ei"], dtype=torch.float),
            "ns": torch.tensor(mbti_label["ns"], dtype=torch.float),
            "tf": torch.tensor(mbti_label["tf"], dtype=torch.float),
            "jp": torch.tensor(mbti_label["jp"], dtype=torch.float),
            "class_id": torch.tensor(class_id, dtype=torch.long),
            "label": item["type"]
        }

# ===================== 自定义 collate：四个视角分别动态 padding =====================
def make_collate_fn(tokenizer, pad_to_multiple_of=8):
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

    def pad_view(batch, ids_key, mask_key):
        feats = [{"input_ids": b[ids_key], "attention_mask": b[mask_key]} for b in batch]
        padded = tokenizer.pad(
            feats,
            padding="longest",
            return_tensors="pt",
            pad_to_multiple_of=pad_to_multiple_of
        )
        return padded["input_ids"], padded["attention_mask"]

    def right_pad_to(x, L, value):
        # x: (B, Lx) -> (B, L) 右填充
        if x.size(1) == L: 
            return x
        return F.pad(x, (0, L - x.size(1)), value=value)

    def collate(batch):
        # 四个视角分别 pad（到各自视角的最长）
        base_ids, base_m = pad_view(batch, "input_ids",      "attention_mask")
        sem_ids,  sem_m  = pad_view(batch, "semantic_ids",   "semantic_mask")
        sen_ids,  sen_m  = pad_view(batch, "sentiment_ids",  "sentiment_mask")
        lin_ids,  lin_m  = pad_view(batch, "linguistic_ids", "linguistic_mask")

        # 统一到同一个 max_len（四个视角的最大）
        L = max(base_ids.size(1), sem_ids.size(1), sen_ids.size(1), lin_ids.size(1))
        base_ids = right_pad_to(base_ids, L, pad_id);   base_m = right_pad_to(base_m, L, 0)
        sem_ids  = right_pad_to(sem_ids,  L, pad_id);   sem_m  = right_pad_to(sem_m,  L, 0)
        sen_ids  = right_pad_to(sen_ids,  L, pad_id);   sen_m  = right_pad_to(sen_m,  L, 0)
        lin_ids  = right_pad_to(lin_ids,  L, pad_id);   lin_m  = right_pad_to(lin_m,  L, 0)

        return {
            "input_ids":      base_ids, "attention_mask":      base_m,
            "semantic_ids":   sem_ids,  "semantic_mask":       sem_m,
            "sentiment_ids":  sen_ids,  "sentiment_mask":      sen_m,
            "linguistic_ids": lin_ids,  "linguistic_mask":     lin_m,
            "ei": torch.stack([b["ei"] for b in batch]).float(),
            "ns": torch.stack([b["ns"] for b in batch]).float(),
            "tf": torch.stack([b["tf"] for b in batch]).float(),
            "jp": torch.stack([b["jp"] for b in batch]).float(),
            "class_id": torch.stack([b["class_id"] for b in batch]).long(),
            "label": [b["label"] for b in batch],
        }
    return collate

# ===================== 模型 =====================
class JointMBTIModel(nn.Module):
    def __init__(self, encoder_name=MODEL_NAME, dropout=0.3):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name)
        self.encoder.gradient_checkpointing_enable(
    gradient_checkpointing_kwargs={"use_reentrant": False}
)
        hidden = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        self.classifier_ei = nn.Linear(hidden, 1)
        self.classifier_ns = nn.Linear(hidden, 1)
        self.classifier_tf = nn.Linear(hidden, 1)
        self.classifier_jp = nn.Linear(hidden, 1)

    def get_embedding(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls = self.dropout(out.last_hidden_state[:, 0, :])
        return cls

    def forward_heads(self, cls_emb):
        return {
            "embedding": cls_emb,
            "ei": self.classifier_ei(cls_emb),
            "ns": self.classifier_ns(cls_emb),
            "tf": self.classifier_tf(cls_emb),
            "jp": self.classifier_jp(cls_emb)
        }

# ===================== 损失 =====================
def compute_mbti_loss(preds, labels):
    bce = nn.BCEWithLogitsLoss()
    return (
        bce(preds["ei"].squeeze(), labels["ei"]) +
        bce(preds["ns"].squeeze(), labels["ns"]) +
        bce(preds["tf"].squeeze(), labels["tf"]) +
        bce(preds["jp"].squeeze(), labels["jp"])
    ) / 4

def supervised_contrastive_loss(embeddings, labels, temperature=0.07):
    # embeddings: (N, H) ; labels: (N,)
    device = embeddings.device
    z = torch.nn.functional.normalize(embeddings, dim=1)
    sim = torch.matmul(z, z.T) / temperature  # (N,N)

    eye = torch.eye(len(labels), dtype=torch.bool, device=device)
    labels = labels.view(-1, 1)
    match = torch.eq(labels, labels.T).to(device) & ~eye

    logits_max, _ = sim.max(dim=1, keepdim=True)
    logits = sim - logits_max.detach()
    exp_logits = torch.exp(logits) * (~eye)
    log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

    mean_log_prob_pos = (match * log_prob).sum(1) / match.sum(1).clamp(min=1)
    return -(mean_log_prob_pos.mean())

# ===================== 评估 =====================
@torch.no_grad()
def evaluate(model, dataloader, device):
    model.eval()
    total = 0
    correct_ei = correct_ns = correct_tf = correct_jp = correct_all = 0
    for batch in dataloader:
        base_ids = batch["input_ids"].to(device)
        base_msk = batch["attention_mask"].to(device)
        cls = model.get_embedding(base_ids, base_msk)
        out = model.forward_heads(cls)

        labels = {k: batch[k].to(device) for k in ["ei","ns","tf","jp"]}
        pred_ei = torch.sigmoid(out["ei"]).round().squeeze()
        pred_ns = torch.sigmoid(out["ns"]).round().squeeze()
        pred_tf = torch.sigmoid(out["tf"]).round().squeeze()
        pred_jp = torch.sigmoid(out["jp"]).round().squeeze()

        correct_ei += (pred_ei == labels["ei"]).sum().item()
        correct_ns += (pred_ns == labels["ns"]).sum().item()
        correct_tf += (pred_tf == labels["tf"]).sum().item()
        correct_jp += (pred_jp == labels["jp"]).sum().item()
        correct_all += ((pred_ei == labels["ei"]) &
                        (pred_ns == labels["ns"]) &
                        (pred_tf == labels["tf"]) &
                        (pred_jp == labels["jp"])).sum().item()
        total += base_ids.size(0)

    return {
        "acc_ei": correct_ei / total,
        "acc_ns": correct_ns / total,
        "acc_tf": correct_tf / total,
        "acc_jp": correct_jp / total,
        "acc_all": correct_all / total
    }

# ===================== 训练 =====================
def train_model(model, train_loader, val_loader, optimizer, device, epochs=5, save_path="best.pt"):
    model.to(device)
    scaler = amp.GradScaler("cuda") if device.type == "cuda" else None

    total_updates = (len(train_loader) // GRAD_ACCUM) * epochs
    warmup_steps  = max(1, int(total_updates * WARMUP_RATIO))
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_updates
    )

    best_all = 0.0
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        optimizer.zero_grad(set_to_none=True)

        for step, batch in enumerate(tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{epochs}")):
            # ---- 四视角合并为一次前向 ----
            base_ids = batch["input_ids"].to(device)
            base_msk = batch["attention_mask"].to(device)
            sem_ids  = batch["semantic_ids"].to(device)
            sem_msk  = batch["semantic_mask"].to(device)
            sen_ids  = batch["sentiment_ids"].to(device)
            sen_msk  = batch["sentiment_mask"].to(device)
            lin_ids  = batch["linguistic_ids"].to(device)
            lin_msk  = batch["linguistic_mask"].to(device)

            all_ids = torch.cat([base_ids, sem_ids, sen_ids, lin_ids], dim=0)   # (4B, L)
            all_msk = torch.cat([base_msk, sem_msk, sen_msk, lin_msk], dim=0)
            B = base_ids.size(0)

            labels_bin = {k: batch[k].to(device) for k in ["ei","ns","tf","jp"]}
            class_id   = batch["class_id"].to(device)

            ctx = amp.autocast("cuda") if device.type == "cuda" else contextlib.nullcontext()
            with ctx:
                emb_all  = model.get_embedding(all_ids, all_msk)      # (4B, H)
                emb_base = emb_all[:B]
                heads    = model.forward_heads(emb_base)

                loss_cls = compute_mbti_loss(heads, labels_bin)
                all_labels = class_id.repeat(4)                       # (4B,)
                loss_cl  = supervised_contrastive_loss(emb_all, all_labels)

                loss = loss_cls + LAMBDA_CL * loss_cl

            loss = loss / GRAD_ACCUM

            if scaler:
                scaler.scale(loss).backward()
                if (step + 1) % GRAD_ACCUM == 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    scaler.step(optimizer); scaler.update()
                    optimizer.zero_grad(set_to_none=True)
                    scheduler.step()
            else:
                loss.backward()
                if (step + 1) % GRAD_ACCUM == 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step(); optimizer.zero_grad(set_to_none=True)
                    scheduler.step()

            total_loss += loss.item() * GRAD_ACCUM  # 还原显示

        avg_loss = total_loss / len(train_loader)
        print(f"\nEpoch {epoch+1} - Train Loss: {avg_loss:.4f}")

        metrics = evaluate(model, val_loader, device)
        print(f"Validation | EI: {metrics['acc_ei']:.2%} NS: {metrics['acc_ns']:.2%} TF: {metrics['acc_tf']:.2%} JP: {metrics['acc_jp']:.2%} 4D: {metrics['acc_all']:.2%}")

        if metrics["acc_all"] > best_all:
            best_all = metrics["acc_all"]
            torch.save(model.state_dict(), save_path)
            print(f"✅ Best model saved to {save_path} (4D: {best_all:.2%})")

    model.eval()

# ===================== 主流程 =====================
if __name__ == "__main__":
    # 读数据 & 划分
    with open(FILE_PATH, "r", encoding="utf-8") as f:
        all_data = json.load(f)
    train_data, val_data = train_test_split(all_data, test_size=0.1, random_state=42)

    # tokenizer & 自定义 collate（四视角分别动态 padding）
    tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)
    collate_fn = make_collate_fn(tokenizer, pad_to_multiple_of=8)

    train_ds = JointMBTIDataset(train_data, tokenizer, max_length=MAX_LEN, drop_prob=0.1)
    val_ds   = JointMBTIDataset(val_data,   tokenizer, max_length=MAX_LEN, drop_prob=0.0)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=collate_fn, num_workers=0, pin_memory=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=collate_fn, num_workers=0, pin_memory=True)

    # 模型 & 优化器
    model = JointMBTIModel(MODEL_NAME)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

    # 训练
    train_model(model, train_loader, val_loader, optimizer, DEVICE, epochs=EPOCHS, save_path=SAVE_PATH)

    # 推理示例（主文本）
    sample = val_data[0]
    enc = tokenizer(sample.get("posts_cleaned", sample.get("posts","")),
                    truncation=True, max_length=MAX_LEN, return_tensors="pt")
    with torch.no_grad():
        cls = model.get_embedding(enc["input_ids"].to(DEVICE), enc["attention_mask"].to(DEVICE))
        out = model.forward_heads(cls)
        pred_ei = torch.sigmoid(out["ei"]).round().item()
        pred_ns = torch.sigmoid(out["ns"]).round().item()
        pred_tf = torch.sigmoid(out["tf"]).round().item()
        pred_jp = torch.sigmoid(out["jp"]).round().item()
    print("原标签:", sample["type"])
    print("预测四维:", pred_ei, pred_ns, pred_tf, pred_jp)
# ====================== 结束 ======================


[Train] Epoch 1/30: 100%|██████████| 3600/3600 [21:07<00:00,  2.84it/s]



Epoch 1 - Train Loss: 1.0725
Validation | EI: 62.60% NS: 63.54% TF: 67.90% JP: 60.35% 4D: 19.75%
✅ Best model saved to best_deberta_supcon.pt (4D: 19.75%)


[Train] Epoch 2/30: 100%|██████████| 3600/3600 [20:45<00:00,  2.89it/s]



Epoch 2 - Train Loss: 0.9754
Validation | EI: 64.81% NS: 66.81% TF: 63.31% JP: 65.52% 4D: 21.62%
✅ Best model saved to best_deberta_supcon.pt (4D: 21.62%)


[Train] Epoch 3/30: 100%|██████████| 3600/3600 [20:46<00:00,  2.89it/s]


...


Epoch 9 - Train Loss: 0.5659
Validation | EI: 72.62% NS: 75.33% TF: 75.60% JP: 72.94% 4D: 39.33%
✅ Best model saved to best_deberta_supcon.pt (4D: 39.33%)


[Train] Epoch 10/30: 100%|██████████| 3600/3600 [20:46<00:00,  2.89it/s]



Epoch 10 - Train Loss: 0.5256
Validation | EI: 73.17% NS: 74.54% TF: 75.38% JP: 71.04% 4D: 38.96%


[Train] Epoch 11/30:  42%|████▏     | 1497/3600 [08:38<12:08,  2.89it/s]


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-
"""
Meta-Llama-3-8B-Instruct + QLoRA(4bit) + LoRA
任务：MBTI 16类分类（同时统计4D严格准确率）
适配 Transformers==4.55（使用 eval_strategy）
"""

import os, json
from typing import Dict, Any, List

# —— 关键：禁用 accelerate 混合精度，避免对量化模型做 .to(dtype) —— #
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

import numpy as np
import torch
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    DataCollatorWithPadding,
    Trainer, TrainingArguments,
    set_seed,
)

# 可选：启用 flash-attn-2（未安装会自动忽略）
try:
    from transformers import set_attn_implementation
    set_attn_implementation("flash_attention_2")
except Exception:
    pass

# ============ 配置 ============
MODEL_NAME   = "meta-llama/Meta-Llama-3-8B-Instruct"
FILE_PATH    = "mbti_sample_with_all_views_pandora.json"

# 序列预算（总长约 768）
MAX_LEN      = 768
BUDGET = {"posts_cleaned": 384, "semantic_view": 128, "sentiment_view": 128, "linguistic_view": 128}

SEED         = 42
EPOCHS       = 4
LR           = 2e-4
BSZ_TRN      = 16
BSZ_EVAL     = 32
GRAD_ACCUM   = 2
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01
OUTPUT_DIR   = "mbti_lora_llama8b_ckpt"

# QLoRA & LoRA
USE_4BIT     = True
LORA_R       = 16
LORA_ALPHA   = 32
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

# HF token（如有门控模型），建议 export HF_TOKEN=xxx
HF_TOKEN = os.getenv("HF_TOKEN")
HF_KW = {"token": HF_TOKEN} if HF_TOKEN else {}

# ============ MBTI 工具 ============
MBTI_16 = [
    "INTJ","INTP","ENTJ","ENTP","INFJ","INFP","ENFJ","ENFP",
    "ISTJ","ISFJ","ESTJ","ESFJ","ISTP","ISFP","ESTP","ESFP"
]
MBTI2ID = {t:i for i,t in enumerate(MBTI_16)}

def mbti_to_4d(m: str):
    # I/E, S/N, F/T, P/J -> 0/1
    return (
        0 if m[0]=="I" else 1,
        0 if m[1]=="S" else 1,
        0 if m[2]=="F" else 1,
        0 if m[3]=="P" else 1,
    )

def load_rows(path: str):
    with open(path, "r", encoding="utf-8") as f:
        rows = json.load(f)
    return [r for r in rows if r.get("type") in MBTI2ID]

def truncate_to_budget(tok: AutoTokenizer, text: str, budget: int) -> str:
    enc = tok(text or "", add_special_tokens=False)
    ids = enc["input_ids"][: budget]
    return tok.decode(ids)

def build_input(item: Dict[str, Any], tok: AutoTokenizer) -> str:
    p   = truncate_to_budget(tok, item.get("posts_cleaned", item.get("posts","")) or "", BUDGET["posts_cleaned"])
    sem = truncate_to_budget(tok, item.get("semantic_view","")  or "", BUDGET["semantic_view"])
    sen = truncate_to_budget(tok, item.get("sentiment_view","") or "", BUDGET["sentiment_view"])
    lin = truncate_to_budget(tok, item.get("linguistic_view","") or "", BUDGET["linguistic_view"])
    return (
        f"[POSTS]\n{p}\n[SEMANTIC]\n{sem}\n[SENTIMENT]\n{sen}\n[LINGUISTIC]\n{lin}\n"
        f"[TASK] Predict MBTI type among {', '.join(MBTI_16)}."
    )

# ============ Dataset ============
class MBTIDataset(torch.utils.data.Dataset):
    def __init__(self, rows, tokenizer, max_len=512):
        self.rows = rows
        self.tok  = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.rows)
    def __getitem__(self, idx):
        it  = self.rows[idx]
        text= build_input(it, self.tok)
        y   = MBTI2ID[it["type"]]
        enc = self.tok(text, truncation=True, max_length=self.max_len)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": y}

# ============ 指标 ============
def compute_metrics(eval_pred):
    # 兼容 transformers 各版本返回：可能是 (preds, labels) 或 EvalPrediction
    if isinstance(eval_pred, tuple):
        preds, labels = eval_pred
    else:
        preds, labels = eval_pred.predictions, eval_pred.label_ids

    # preds 可能是 (logits,) 或 numpy
    if isinstance(preds, (list, tuple)):
        preds = preds[0]
    if not isinstance(preds, np.ndarray):
        preds = np.asarray(preds)
    if not isinstance(labels, np.ndarray):
        labels = np.asarray(labels)

    pred_ids = preds.argmax(-1)
    acc16 = float((pred_ids == labels).mean())

    pred_types = [MBTI_16[i] for i in pred_ids]
    true_types = [MBTI_16[i] for i in labels]

    c_ei=c_ns=c_tf=c_jp=c_all=0
    for pt, tt in zip(pred_types, true_types):
        pei,pns,ptf,pjp = mbti_to_4d(pt)
        tei,tns,ttf,tjp = mbti_to_4d(tt)
        c_ei += (pei==tei); c_ns += (pns==tns); c_tf += (ptf==ttf); c_jp += (pjp==tjp)
        c_all+= (pei==tei and pns==tns and ptf==ttf and pjp==tjp)
    n = len(labels)
    return {"acc_16": acc16, "acc_ei": c_ei/n, "acc_ns": c_ns/n, "acc_tf": c_tf/n, "acc_jp": c_jp/n, "acc_4D": c_all/n}

# ============ 主流程 ============
def main():
    set_seed(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # 数据
    rows = load_rows(FILE_PATH)
    train_rows, val_rows = train_test_split(
        rows, test_size=0.1, random_state=SEED, stratify=[r["type"] for r in rows]
    )

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, **HF_KW)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # QLoRA 量化配置（注意：不要再传 torch_dtype）
    quant_cfg = BitsAndBytesConfig(
        load_in_4bit=USE_4BIT,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,  # 稳定优先；新 bnb 版本也可换 torch.bfloat16
    ) if USE_4BIT else None

    # 模型（16类分类头）
    model_kwargs = dict(
        num_labels=16,
        quantization_config=quant_cfg,
        device_map="auto",
        **HF_KW,
    )
    if not USE_4BIT:
        model_kwargs["torch_dtype"] = torch.bfloat16

    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, **model_kwargs)

    # —— 保险：禁止任何对量化模型的 .to() —— #
    def _noop_to(self, *args, **kwargs):
        return self
    model.to = _noop_to.__get__(model, type(model))

    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.use_cache = False
    model.resize_token_embeddings(len(tokenizer))

    # LoRA（含 k-bit 训练准备）
    from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
    peft_cfg = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=LORA_R, lora_alpha=LORA_ALPHA, lora_dropout=LORA_DROPOUT,
        target_modules=TARGET_MODULES, bias="none"
    )
    model = get_peft_model(model, peft_cfg)
    model.to = _noop_to.__get__(model, type(model))  # 再补一次

    # 数据 & collator
    train_ds = MBTIDataset(train_rows, tokenizer, max_len=MAX_LEN)
    val_ds   = MBTIDataset(val_rows,   tokenizer, max_len=MAX_LEN)
    collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

    # 训练参数（Transformers 4.55 使用 eval_strategy）
    args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BSZ_TRN,
        per_device_eval_batch_size=BSZ_EVAL,
        gradient_accumulation_steps=GRAD_ACCUM,
        num_train_epochs=EPOCHS,
        learning_rate=LR,
        warmup_ratio=WARMUP_RATIO,
        weight_decay=WEIGHT_DECAY,
        lr_scheduler_type="linear",
        eval_strategy="epoch",      # ← 4.55 新名字
        save_strategy="epoch",
        save_total_limit=2,
        logging_steps=50,
        bf16=False,                 # 量化 + 混合精度容易触发 cast，统一关闭
        fp16=False,
        report_to="none",
        load_best_model_at_end=True,
        metric_for_best_model="eval_acc_4D",
        greater_is_better=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_metrics = trainer.evaluate()
    print("\n=== Final Eval ===")
    for k, v in eval_metrics.items():
        try:
            print(f"{k}: {float(v):.4f}")
        except Exception:
            print(k, v)

    trainer.save_model(OUTPUT_DIR)
    print(f"\n✅ LoRA adapter saved to: {OUTPUT_DIR}")

    # 简单推理示例（只移动 inputs 到 model.device）
    model.eval()
    sample = val_rows[0]
    text = build_input(sample, tokenizer)
    batch = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LEN)
    batch = {k: v.to(next(model.parameters()).device) for k, v in batch.items()}
    with torch.no_grad():
        logits = model(**batch).logits
        pred_id = int(torch.argmax(logits, dim=-1))
        pred_mbti = MBTI_16[pred_id]
    print("\n原标签:", sample["type"], " | 预测:", pred_mbti)

if __name__ == "__main__":
    main()


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.88s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_126962/3440939658.py:231: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/hli962/miniconda3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the 

Epoch,Training Loss,Validation Loss


In [2]:
import sys, os, transformers, accelerate, peft
print("PYTHON:", sys.executable)
print("transformers:", transformers.__version__, "->", os.path.dirname(transformers.__file__))
print("accelerate:", accelerate.__version__)
print("peft:", peft.__version__)


PYTHON: /home/hli962/miniconda3/bin/python
transformers: 4.55.4 -> /home/hli962/miniconda3/lib/python3.12/site-packages/transformers
accelerate: 1.10.1
peft: 0.17.1


In [4]:
import sys, transformers
print(sys.executable)
print("transformers:", transformers.__version__, "->", transformers.__file__)


/home/hli962/miniconda3/bin/python
transformers: 4.55.4 -> /home/hli962/miniconda3/lib/python3.12/site-packages/transformers/__init__.py


In [5]:
# -*- coding: utf-8 -*-
"""
Qwen-2.5-1.5B + LoRA 训练 MBTI 16类，同时统计4D严格准确率
（不做量化，避免 .to() / bitsandbytes 的兼容问题；适配 transformers 4.55+）
"""

import os, json
from typing import Dict, Any, List

import numpy as np
import torch
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer, TrainingArguments,
    set_seed,
)

# ---------------- 基本配置 ----------------
MODEL_NAME = "Qwen/Qwen2.5-1.5B"              # 也可用 "Qwen/Qwen2.5-1.5B-Instruct"
FILE_PATH  = "mbti_sample_with_all_views_pandora.json"

MAX_LEN = 768
BUDGET = {
    "posts_cleaned": 384,
    "semantic_view": 128,
    "sentiment_view": 128,
    "linguistic_view": 128,
}

SEED         = 42
EPOCHS       = 4
LR           = 2e-4
BSZ_TRN      = 4           # 显存吃紧就再降
BSZ_EVAL     = 8
GRAD_ACCUM   = 2
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01
OUTPUT_DIR   = "mbti_qwen_lora_ckpt"

# ---------------- MBTI 工具 ----------------
MBTI_16 = [
    "INTJ","INTP","ENTJ","ENTP",
    "INFJ","INFP","ENFJ","ENFP",
    "ISTJ","ISFJ","ESTJ","ESFJ",
    "ISTP","ISFP","ESTP","ESFP",
]
MBTI2ID = {t:i for i,t in enumerate(MBTI_16)}

def mbti_to_4d(m: str):
    # I/E, S/N, F/T, P/J -> 0/1
    return (
        0 if m[0]=="I" else 1,
        0 if m[1]=="S" else 1,
        0 if m[2]=="F" else 1,
        0 if m[3]=="P" else 1,
    )

def truncate_to_budget(tok: AutoTokenizer, text: str, budget: int) -> str:
    enc = tok(text or "", add_special_tokens=False)
    ids = enc["input_ids"][: budget]
    return tok.decode(ids)

def build_input(item: Dict[str, Any], tok: AutoTokenizer) -> str:
    p   = truncate_to_budget(tok, item.get("posts_cleaned", item.get("posts","")) or "", BUDGET["posts_cleaned"])
    sem = truncate_to_budget(tok, item.get("semantic_view","")  or "", BUDGET["semantic_view"])
    sen = truncate_to_budget(tok, item.get("sentiment_view","") or "", BUDGET["sentiment_view"])
    lin = truncate_to_budget(tok, item.get("linguistic_view","") or "", BUDGET["linguistic_view"])
    return (
        f"[POSTS]\n{p}\n"
        f"[SEMANTIC]\n{sem}\n"
        f"[SENTIMENT]\n{sen}\n"
        f"[LINGUISTIC]\n{lin}\n"
        f"[TASK] Predict MBTI type among {', '.join(MBTI_16)}."
    )

def load_rows(path: str) -> List[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        rows = json.load(f)
    return [r for r in rows if r.get("type") in MBTI2ID]

# ---------------- Dataset ----------------
class MBTIDataset(torch.utils.data.Dataset):
    def __init__(self, rows, tokenizer, max_len=512):
        self.rows = rows
        self.tok  = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.rows)
    def __getitem__(self, idx):
        it  = self.rows[idx]
        text= build_input(it, self.tok)
        y   = MBTI2ID[it["type"]]
        enc = self.tok(text, truncation=True, max_length=self.max_len)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": y}

# ---------------- 指标 ----------------
def compute_metrics(eval_pred):
    # 兼容 EvalPrediction / (preds, labels) / logits tuple
    preds, labels = (eval_pred.predictions, eval_pred.label_ids) if hasattr(eval_pred, "predictions") else eval_pred
    if isinstance(preds, tuple):          # 有些模型返回 (logits, ...)
        preds = preds[0]
    if isinstance(preds, torch.Tensor):
        preds = preds.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu().numpy()

    preds = preds.argmax(-1)
    acc16 = float((preds == labels).mean())

    pred_types = [MBTI_16[i] for i in preds]
    true_types = [MBTI_16[i] for i in labels]

    c_ei=c_ns=c_tf=c_jp=c_all=0
    for pt, tt in zip(pred_types, true_types):
        pei,pns,ptf,pjp = mbti_to_4d(pt)
        tei,tns,ttf,tjp = mbti_to_4d(tt)
        c_ei += (pei==tei); c_ns += (pns==tns); c_tf += (ptf==ttf); c_jp += (pjp==tjp)
        c_all+= (pei==tei and pns==tns and ptf==ttf and pjp==tjp)
    n = len(labels)
    return {
        "acc_16": acc16,
        "acc_ei": c_ei/n, "acc_ns": c_ns/n, "acc_tf": c_tf/n, "acc_jp": c_jp/n,
        "acc_4D": c_all/n,
    }

# ---------------- 训练主流程 ----------------
def main():
    set_seed(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    rows = load_rows(FILE_PATH)
    train_rows, val_rows = train_test_split(
        rows, test_size=0.1, random_state=SEED, stratify=[r["type"] for r in rows]
    )

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # 模型（不量化）+ 分类头
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=16,
        torch_dtype=(torch.bfloat16 if torch.cuda.is_available() else None),
        device_map="auto",
    )
    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.use_cache = False
    model.resize_token_embeddings(len(tokenizer))
    if hasattr(model, "gradient_checkpointing_enable"):
        model.gradient_checkpointing_enable()

    # 只做 LoRA（不做 k-bit）
    from peft import LoraConfig, TaskType, get_peft_model
    peft_cfg = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=16, lora_alpha=32, lora_dropout=0.05,
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
        bias="none"
    )
    model = get_peft_model(model, peft_cfg)
    model.print_trainable_parameters()

    train_ds = MBTIDataset(train_rows, tokenizer, max_len=MAX_LEN)
    val_ds   = MBTIDataset(val_rows,   tokenizer, max_len=MAX_LEN)
    collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

    # ✅ transformers 4.55+ 使用 eval_strategy
    args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BSZ_TRN,
        per_device_eval_batch_size=BSZ_EVAL,
        gradient_accumulation_steps=GRAD_ACCUM,
        num_train_epochs=EPOCHS,
        learning_rate=LR,
        warmup_ratio=WARMUP_RATIO,
        weight_decay=WEIGHT_DECAY,
        logging_steps=50,
        save_total_limit=2,
        report_to="none",
        load_best_model_at_end=True,
        metric_for_best_model="eval_acc_4D",
        greater_is_better=True,
        eval_strategy="epoch",      # <-- 新参数名
        save_strategy="epoch",
        bf16=(torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8),
        fp16=False,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_metrics = trainer.evaluate()
    print("\n=== Final Eval ===")
    for k, v in eval_metrics.items():
        try:
            print(f"{k}: {float(v):.4f}")
        except Exception:
            print(k, v)

    trainer.save_model(OUTPUT_DIR)
    print(f"\n✅ LoRA adapter saved to: {OUTPUT_DIR}")

    # 简单推理示例
    model.eval()
    sample = val_rows[0]
    text = build_input(sample, tokenizer)
    batch = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LEN)
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        logits = model(**batch).logits
        pred_id = int(torch.argmax(logits, dim=-1))
        pred_mbti = MBTI_16[pred_id]
    print("\n原标签:", sample["type"], "| 预测:", pred_mbti)

if __name__ == "__main__":
    main()


Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 18,489,344 || all params: 1,561,811,968 || trainable%: 1.1838


/tmp/ipykernel_126962/1371604356.py:196: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Acc 16,Acc Ei,Acc Ns,Acc Tf,Acc Jp,Acc 4d
1,4.080800,2.028699,0.357083,0.667083,0.695417,0.705208,0.676458,0.357083
2,3.039500,1.809326,0.453333,0.731667,0.749167,0.746458,0.709583,0.453333
3,1.760800,2.001467,0.475833,0.743125,0.752292,0.760208,0.726458,0.475833


/home/hli962/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/hli962/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/hli962/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


KeyboardInterrupt: 

In [2]:
import json
from transformers import AutoTokenizer
from torch.optim import AdamW
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# ===== 1. 读取数据 =====
with open("mbti_sample_with_all_views.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# ===== 2. 划分训练/验证集 =====
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# ===== 3. 初始化Tokenizer和Dataset =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = JointMBTIDataset(train_data, tokenizer, max_length=MAX_LEN, drop_prob=0.1)
val_dataset = JointMBTIDataset(val_data, tokenizer, max_length=MAX_LEN, drop_prob=0.0)  # 验证集不drop

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ===== 4. 初始化模型和优化器 =====
model = JointMBTIModel(encoder_name=MODEL_NAME)
optimizer = AdamW(model.parameters(), lr=LR)

# ===== 5. 开始训练 =====
train_model(model, train_loader, val_loader, optimizer, DEVICE, epochs=EPOCHS, save_path=SAVE_PATH)

# ===== 6. 加载最佳模型做推理 =====
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()

# 推理示例
sample = val_data[0]
encoding = tokenizer(
    sample["posts_cleaned"],
    padding="max_length",
    truncation=True,
    max_length=MAX_LEN,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = model(encoding["input_ids"].to(DEVICE), encoding["attention_mask"].to(DEVICE))
    pred_ei = torch.sigmoid(outputs["ei"]).round().item()
    pred_ns = torch.sigmoid(outputs["ns"]).round().item()
    pred_tf = torch.sigmoid(outputs["tf"]).round().item()
    pred_jp = torch.sigmoid(outputs["jp"]).round().item()

print("原标签:", sample["type"])
print("预测四维:", pred_ei, pred_ns, pred_tf, pred_jp)


NameError: name 'JointMBTIDataset' is not defined

In [1]:
# pip install transformers==4.43.0  (若未安装)
from transformers import AutoTokenizer
import json, os, math
from statistics import mean, median

# === 配置区 ===
FILE_PATH = r"mbti_sample_with_all_views_pandora.json"  # ← 改成你的文件路径
MODEL_NAME = "microsoft/deberta-v3-base"  # 你在用哪种模型就换成对应 tokenizer

# === 载入 tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# === 读入 JSON ===
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"文件不存在：{FILE_PATH}")

with open(FILE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# data 预期是一个 list，每个元素是 dict，包含 "posts_cleaned"
if not isinstance(data, list):
    raise ValueError("JSON 顶层应为 list，每个元素为一条样本。")

def count_tokens(text: str) -> int:
    # add_special_tokens=True 表示会包含 [CLS]/[SEP] 等特殊符号（和训练/推理一致）
    enc = tokenizer(text, add_special_tokens=True, truncation=False)
    return len(enc["input_ids"])

# === 计算前 10 条 ===
first10 = data[:10]
lens = []
print("—— 前 10 条 posts_cleaned 的 token 数 ——")
for i, item in enumerate(first10, 1):
    text = item.get("posts_cleaned", "")
    tok_len = count_tokens(text)
    lens.append(tok_len)
    print(f"样本 {i:02d}: {tok_len} tokens")

# === 给出整体统计（可选：只对前10统计，或你也可以改成对全量统计）===
def pctl(arr, p):
    arr = sorted(arr)
    if not arr:
        return 0
    k = (len(arr)-1) * p
    f = math.floor(k)
    c = math.ceil(k)
    if f == c:
        return arr[int(k)]
    return arr[f] + (arr[c] - arr[f]) * (k - f)

print("\n—— 前 10 条统计 ——")
print(f"均值: {mean(lens):.1f}")
print(f"中位数: {median(lens):.1f}")
print(f"P90: {pctl(lens, 0.90):.1f}")
print(f"最大值: {max(lens)}")
print(f"最小值: {min(lens)}")


/home/hli962/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hli962/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


—— 前 10 条 posts_cleaned 的 token 数 ——
样本 01: 244 tokens
样本 02: 233 tokens
样本 03: 276 tokens
样本 04: 243 tokens
样本 05: 271 tokens
样本 06: 269 tokens
样本 07: 207 tokens
样本 08: 299 tokens
样本 09: 239 tokens
样本 10: 261 tokens

—— 前 10 条统计 ——
均值: 254.2
中位数: 252.5
P90: 278.3
最大值: 299
最小值: 207


In [1]:
import json, os, math, numpy as np
from statistics import mean, median
from transformers import AutoTokenizer

# ===== 配置（按你当前训练的真实值填写）=====
MODEL_NAME = "microsoft/deberta-v3-base"
FILE_PATH  = "mbti_sample_with_all_views_pandora.json"

BASE_MAX_LEN   = 512   # 你当前/之前用的 max_length（若原来512就填512）
BASE_BATCHSIZE = 8     # 你当前/之前用的 batch_size
TARGET_TOKENS_PER_STEP = BASE_MAX_LEN * BASE_BATCHSIZE  # 保持每步 token 数不变

# ===== 读取数据 & tokenizer =====
assert os.path.exists(FILE_PATH), f"文件不存在: {FILE_PATH}"
with open(FILE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def count_tokens(text: str) -> int:
    enc = tokenizer(text, add_special_tokens=True, truncation=False)
    return len(enc["input_ids"])

# ===== 全量长度统计 =====
lengths = [count_tokens(x.get("posts_cleaned", "")) for x in data]
p90 = float(np.percentile(lengths, 90))
p95 = float(np.percentile(lengths, 95))
avg = mean(lengths)
med = median(lengths)
mx  = max(lengths)
mn  = min(lengths)

print("—— 全量 posts_cleaned token 统计 ——")
print(f"样本数: {len(lengths)}")
print(f"均值: {avg:.1f}, 中位数: {med:.1f}, P90: {p90:.1f}, P95: {p95:.1f}, 最小: {mn}, 最大: {mx}")

# ===== 建议的 MAX_LEN（P90 + 余量，并对齐到32）=====
def round_up_to_multiple(x, base=32):
    return int(base * math.ceil(x / base))

suggest_len = round_up_to_multiple(p90 + 16, 32)  # 16 作为小余量
# 你也可以限制范围避免过长/过短，比如：
suggest_len = min(max(suggest_len, 224), 448)

print(f"\n建议 MAX_LEN: {suggest_len}  (依据 P90≈{p90:.0f} 推出)")

# ===== 建议的 batch_size / 梯度累积 =====
new_bs = max(1, TARGET_TOKENS_PER_STEP // suggest_len)
tokens_per_step = new_bs * suggest_len
print(f"建议 batch_size: {new_bs}  (保持每步 token≈{TARGET_TOKENS_PER_STEP} → 当前≈{tokens_per_step})")

# 如果你希望“至少达到”目标 token/step，可以给出梯度累积建议：
if tokens_per_step < TARGET_TOKENS_PER_STEP:
    need = math.ceil(TARGET_TOKENS_PER_STEP / tokens_per_step)
    print(f"可选：设置梯度累积 grad_accum={need}，使有效 tokens/step ≥ 之前水平。")

# =====（可选）看看如果更激进用 288/320 会怎样 =====
for test_len in [288, 320]:
    test_bs = max(1, TARGET_TOKENS_PER_STEP // test_len)
    print(f"\n【对照】max_len={test_len}: 建议 batch_size={test_bs}, "
          f"tokens/step≈{test_bs*test_len}, 截断比例估计≈{sum(l>test_len for l in lengths)/len(lengths):.1%}")


/home/hli962/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hli962/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


—— 全量 posts_cleaned token 统计 ——
样本数: 48000
均值: 237.1, 中位数: 239.0, P90: 284.0, P95: 307.0, 最小: 3, 最大: 1781

建议 MAX_LEN: 320  (依据 P90≈284 推出)
建议 batch_size: 12  (保持每步 token≈4096 → 当前≈3840)
可选：设置梯度累积 grad_accum=2，使有效 tokens/step ≥ 之前水平。

【对照】max_len=288: 建议 batch_size=14, tokens/step≈4032, 截断比例估计≈8.5%

【对照】max_len=320: 建议 batch_size=12, tokens/step≈3840, 截断比例估计≈3.7%


In [ ]:
import json
from transformers import AutoTokenizer
from torch.optim import AdamW
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# ===== 1. 读取数据 =====
with open("mbti_sample_with_all_views_pandora.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# ===== 2. 划分训练/验证集 =====
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# ===== 3. 初始化Tokenizer和Dataset =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = JointMBTIDataset(train_data, tokenizer, max_length=MAX_LEN, drop_prob=0.1)
val_dataset = JointMBTIDataset(val_data, tokenizer, max_length=MAX_LEN, drop_prob=0.0)  # 验证集不drop

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ===== 4. 初始化模型和优化器 =====
model = JointMBTIModel(encoder_name=MODEL_NAME)
optimizer = AdamW(model.parameters(), lr=LR)

# ===== 5. 开始训练 =====
train_model(model, train_loader, val_loader, optimizer, DEVICE, epochs=EPOCHS, save_path=SAVE_PATH)

# ===== 6. 加载最佳模型做推理 =====
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()

# 推理示例
sample = val_data[0]
encoding = tokenizer(
    sample["posts_cleaned"],
    padding="max_length",
    truncation=True,
    max_length=MAX_LEN,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = model(encoding["input_ids"].to(DEVICE), encoding["attention_mask"].to(DEVICE))
    pred_ei = torch.sigmoid(outputs["ei"]).round().item()
    pred_ns = torch.sigmoid(outputs["ns"]).round().item()
    pred_tf = torch.sigmoid(outputs["tf"]).round().item()
    pred_jp = torch.sigmoid(outputs["jp"]).round().item()

print("原标签:", sample["type"])
print("预测四维:", pred_ei, pred_ns, pred_tf, pred_jp)

/home/hli962/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
[Train] Epoch 1/30: 100%|██████████| 10800/10800 [28:42<00:00,  6.27it/s]



Epoch 1 - Train Loss: 0.8944
Validation | EI: 64.12% NS: 63.40% TF: 68.94% JP: 61.23% 4D: 20.48%
✅ Best model saved to best_deberta_supcon.pt (4D: 20.48%)


[Train] Epoch 2/30: 100%|██████████| 10800/10800 [28:41<00:00,  6.27it/s]



Epoch 2 - Train Loss: 0.8023
Validation | EI: 68.58% NS: 66.88% TF: 71.19% JP: 65.77% 4D: 27.15%
✅ Best model saved to best_deberta_supcon.pt (4D: 27.15%)


[Train] Epoch 3/30: 100%|██████████| 10800/10800 [28:41<00:00,  6.27it/s]


...

[Train] Epoch 16/30: 100%|██████████| 10800/10800 [28:41<00:00,  6.27it/s]



Epoch 16 - Train Loss: 0.2612
Validation | EI: 72.96% NS: 74.54% TF: 74.77% JP: 71.23% 4D: 38.92%
✅ Best model saved to best_deberta_supcon.pt (4D: 38.92%)


[Train] Epoch 17/30: 100%|██████████| 10800/10800 [28:41<00:00,  6.27it/s]



Epoch 17 - Train Loss: 0.2581
Validation | EI: 72.15% NS: 73.85% TF: 74.33% JP: 70.44% 4D: 38.38%


[Train] Epoch 18/30:  38%|███▊      | 4112/10800 [10:55<17:44,  6.28it/s]